# Minimal LangGraph Agent for SageMaker Studio

This notebook demonstrates how to build a simple ReAct-style agent using **LangGraph** and **AWS Bedrock**. 

You'll learn how to:
- Configure and connect to Claude models via Bedrock
- Define custom tools that the agent can use
- Build a graph-based agent that reasons and acts
- Test the agent with various queries

**Prerequisites:** Ensure your model is configured in `../CONFIG.txt` before running.

In [ ]:
# Load configuration from CONFIG.txt
from dotenv import load_dotenv
import os

load_dotenv("../CONFIG.txt")

MODEL_ID = os.getenv("MODEL_ID")
REGION = os.getenv("REGION", "us-west-2")

# Derive BASE_MODEL_ID for Claude inference profiles
# us.anthropic.claude-* -> anthropic.claude-*
if MODEL_ID and MODEL_ID.startswith("us.anthropic."):
    BASE_MODEL_ID = MODEL_ID.replace("us.anthropic.", "anthropic.")
else:
    BASE_MODEL_ID = None

print(f"Model:  {MODEL_ID}")
if BASE_MODEL_ID:
    print(f"Base:   {BASE_MODEL_ID}")
print(f"Region: {REGION}")

In [ ]:
import importlib.metadata

packages = [
    "langchain",
    "langchain-core",
    "langgraph",
    "langchain-aws",
    "langchain-mcp-adapters",
    "mcp",
    "httpx",
    "boto3",
]

print("Pre-installed packages:")
print("-" * 50)
for pkg in packages:
    try:
        version = importlib.metadata.version(pkg)
        print(f"{pkg:30} {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{pkg:30} NOT INSTALLED")

In [ ]:
# Install missing packages
%pip install langgraph>=1.0.6 -q

## 2. Imports

Import the required libraries for building our agent:
- **LangChain AWS**: Provides the `ChatBedrockConverse` class for Bedrock integration
- **LangGraph**: Framework for building stateful, graph-based agents
- **LangChain Core**: Message types and tool decorators

In [ ]:
from typing import Literal
from datetime import datetime

from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

print("Imports successful!")

## 3. Define Tools

Tools are functions that the agent can call to perform actions or retrieve information. We use the `@tool` decorator to register them with LangChain.

Here we define two simple tools:
- **get_current_time**: Returns the current date and time
- **add_numbers**: Adds two integers together

The agent will automatically decide when to use these tools based on the user's question.

In [ ]:
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b


tools = [get_current_time, add_numbers]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

## 4. Initialize the LLM

Create a connection to Claude via AWS Bedrock using `ChatBedrockConverse`. This class uses the Bedrock Converse API, which provides a unified interface for all supported models.

Key configuration:
- **model**: The model ID from CONFIG.txt (e.g., `us.anthropic.claude-3-sonnet-20240229-v1:0`)
- **region_name**: AWS region where Bedrock is available
- **temperature**: Controls randomness (0 = deterministic, 1 = creative)

After initialization, we bind our tools to the LLM so it knows what actions are available.

In [ ]:
# Build LLM config - add base_model_id for Claude inference profiles
llm_kwargs = {
    "model": MODEL_ID,
    "region_name": REGION,
    "temperature": 0,
}

if BASE_MODEL_ID:
    llm_kwargs["base_model_id"] = BASE_MODEL_ID

llm = ChatBedrockConverse(**llm_kwargs)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"LLM initialized with {MODEL_ID}!")

## 5. Build the LangGraph Agent

LangGraph allows us to build agents as **directed graphs** where:
- **Nodes** are functions that process state (e.g., call the LLM, execute tools)
- **Edges** define the flow between nodes
- **Conditional edges** route based on the current state

Our agent follows the **ReAct pattern** (Reason + Act):
1. The `agent` node calls the LLM to decide what to do
2. If the LLM requests tool calls, route to the `tools` node
3. The `tools` node executes the requested tools
4. Return to `agent` to process results and continue reasoning
5. When no more tools are needed, end the conversation

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Determine whether to continue to tools or end."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    """Call the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


# Build the graph
graph = StateGraph(MessagesState)

# Add nodes
graph.add_node("agent", call_model)
graph.add_node("tools", ToolNode(tools))

# Add edges
graph.add_edge(START, "agent")
graph.add_conditional_edges("agent", should_continue)
graph.add_edge("tools", "agent")

# Compile
agent = graph.compile()

print("Agent graph compiled successfully!")

## 6. Visualize the Graph (Optional)

LangGraph can render the agent's graph structure as a diagram. This helps visualize how messages flow between nodes and understand the agent's decision-making process.

**Note:** Requires `graphviz` to be installed. If not available, a text representation is shown instead.

In [ ]:
# Display the graph structure (requires graphviz)
try:
    from IPython.display import Image, display
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Graph visualization not available: {e}")
    print("\nGraph structure:")
    print("  START -> agent -> (tools -> agent) | END")

## 7. Run the Agent

Now let's test the agent! The `run_agent` function:
1. Takes a question as input
2. Wraps it with a system message defining the assistant's behavior
3. Invokes the agent graph
4. Returns the final response

The agent will automatically use tools when needed to answer the question.

In [ ]:
def run_agent(question: str):
    """Run the agent with a question and display the response."""
    print(f"Question: {question}")
    print("-" * 50)
    
    result = agent.invoke({
        "messages": [
            SystemMessage(content="You are a helpful assistant. Use tools when needed."),
            HumanMessage(content=question),
        ]
    })
    
    final_message = result["messages"][-1]
    print(f"\nResponse:\n{final_message.content}")
    return result

In [ ]:
# Test: Get current time
result = run_agent("What is the current time?")

In [ ]:
# Test: Math calculation
result = run_agent("What is 42 + 17?")

In [ ]:
# Test: Multiple tools
result = run_agent("What time is it and what is 100 + 200?")

## 9. Query with Sample Manufacturing Data

This section demonstrates how to provide context to the agent. We load sample manufacturing data and ask the agent questions about it.

This pattern is useful for:
- **RAG (Retrieval-Augmented Generation)**: Injecting retrieved documents into prompts
- **In-context learning**: Providing examples or data for the agent to reference
- **Domain-specific Q&A**: Answering questions based on proprietary information

In [ ]:
# Load sample manufacturing data
from load_sample_data import load_manufacturing_data, print_info

manufacturing_text = load_manufacturing_data()
print_info(manufacturing_text)

In [ ]:
# Ask the agent questions about the manufacturing data
def ask_about_data(question: str, context: str):
    """Ask the agent a question with context."""
    prompt = f"""Based on this manufacturing information:

{context}

Question: {question}"""
    return run_agent(prompt)

# Sample questions
ask_about_data("What components are in the Electric Powertrain domain?", manufacturing_text)

In [ ]:
# Another sample question
ask_about_data("What defects have been found and what are their severities?", manufacturing_text)